In [1]:
import cv2 as cv
import os 
import numpy as np

In [2]:
classes = {'buildings':0, 'forest':1, 'glacier':2, 'mountain':3, 'sea':4, 'street':5}

In [3]:
images = []
labels = []

def load_data(path):
    
    for folders in os.listdir(path):
        label = folders
        for files in os.listdir(path + folders + '/'):
            image = cv.imread(path + folders + '/' + files)
            image = cv.resize(image, (100, 100))
            image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
            image = image.astype('float32')/255.0
            images.append(image)
            labels.append(classes[label])
    return images, labels

In [4]:
images, labels = load_data('E:/NMIMS_Semester III/Image Proceesing - ANN, CNN/datasets/Intel Image Classification/seg_train/seg_train/')

In [5]:
images, labels = load_data('E:/NMIMS_Semester III/Image Proceesing - ANN, CNN/datasets/Intel Image Classification/seg_test/seg_test/')

In [6]:
images = np.array(images)

In [7]:
from tensorflow.keras.utils import to_categorical

In [8]:
labels_req = to_categorical(labels)

print(images.shape)
print(labels_req.shape)

(17034, 100, 100, 3)
(17034, 6)


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(images, labels_req, test_size=0.1, random_state=123)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(15330, 100, 100, 3) (1704, 100, 100, 3) (15330, 6) (1704, 6)


In [11]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dense, Activation, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import glorot_uniform

In [12]:
model = Sequential()
model.add(Input((100, 100, 3)))
model.add(Conv2D(16, (3,3), padding='same', strides=1, activation='relu'))
model.add(Conv2D(16, (3,3), padding='same', strides=1, activation='relu'))
model.add(MaxPooling2D((3,3), strides=2))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(32, (3,3), padding='same', strides=1, activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', strides=1, activation='relu'))
model.add(MaxPooling2D((3,3), strides=2))
model.add(BatchNormalization(axis=3))
model.add(Conv2D(128, (3,3), padding='same', strides=1, activation='relu'))
model.add(MaxPooling2D((3,3), strides=2))
model.add(BatchNormalization(axis=3))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(6, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 16)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 49, 49, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 49, 49, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0

In [14]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='rmsprop')

In [15]:
model.fit(X_train, Y_train, validation_split=0.05, epochs=20, batch_size=64)

Train on 14563 samples, validate on 767 samples
Epoch 1/20
14563/14563 [==============================] - 27s 2ms/sample - loss: 1.4888 - accuracy: 0.6532 - val_loss: 14.4888 - val_accuracy: 0.1734
Epoch 2/20
14563/14563 [==============================] - 28s 2ms/sample - loss: 0.8551 - accuracy: 0.7711 - val_loss: 7.1042 - val_accuracy: 0.2712
Epoch 3/20
14563/14563 [==============================] - 45s 3ms/sample - loss: 0.7520 - accuracy: 0.8120 - val_loss: 1.3932 - val_accuracy: 0.7679
Epoch 4/20
14563/14563 [==============================] - 53s 4ms/sample - loss: 0.6595 - accuracy: 0.8436 - val_loss: 0.9081 - val_accuracy: 0.7653
Epoch 5/20
14563/14563 [==============================] - 59s 4ms/sample - loss: 0.6288 - accuracy: 0.8603 - val_loss: 0.7148 - val_accuracy: 0.8605
Epoch 6/20
14563/14563 [==============================] - 58s 4ms/sample - loss: 0.5896 - accuracy: 0.8826 - val_loss: 0.9298 - val_accuracy: 0.8083
Epoch 7/20
14563/14563 [==============================] -

In [16]:
model.evaluate(X_test, Y_test, batch_size=64)

1704/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.7996516546732942, 0.8579812]

In [17]:
model.save('E:/NMIMS_Semester III/Image Proceesing - ANN, CNN/Saved Models/Intel Image Classification/intel_image_class_1.h5')